In [31]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
import time
import random
from fake_useragent import UserAgent
#libs used to do web scraping and sending req

In [70]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.pakwheels.com/used-cars/honda/32"
}

#these are the headers to mimic th human behavior

In [71]:
base_url="https://www.pakwheels.com/used-cars/honda-lahore/201"#URL of the page

In [72]:
from dotenv import load_dotenv#importing
import os

In [73]:
load_dotenv()#load everything from env folder
mongo_url=os.getenv("DB_URL")
proxy_auth=os.getenv("proxy_key")

In [74]:
proxies={
    'http':f'http://{proxy_auth}',
    'https':f'http://{proxy_auth}'
}
#proxies list to save us from blocking 

In [75]:
session=requests.Session()#set session as making request for session 

In [76]:
client=MongoClient(mongo_url)
data_base=client["Honda_cars"]#set database name
collection=data_base["listings"]#set collection/table


In [77]:
def get_all_pages_of_honda():
    
    for page in range(1, 90):  # first 90 pages
        url = f"{base_url}?page={page}&_pjax=%5Bdata-pjax-container%5D"
        print(f"Saving page {page}: {url}")

        r = requests.get(url, headers=headers)

        with open(f"pages/page_{page}.html", "w", encoding="utf-8") as f:
            f.write(r.text)

        time.sleep(1 + 2 * random.random())


In [78]:
get_all_pages_of_honda()

Saving page 1: https://www.pakwheels.com/used-cars/honda-lahore/201?page=1&_pjax=%5Bdata-pjax-container%5D
Saving page 2: https://www.pakwheels.com/used-cars/honda-lahore/201?page=2&_pjax=%5Bdata-pjax-container%5D
Saving page 3: https://www.pakwheels.com/used-cars/honda-lahore/201?page=3&_pjax=%5Bdata-pjax-container%5D
Saving page 4: https://www.pakwheels.com/used-cars/honda-lahore/201?page=4&_pjax=%5Bdata-pjax-container%5D
Saving page 5: https://www.pakwheels.com/used-cars/honda-lahore/201?page=5&_pjax=%5Bdata-pjax-container%5D
Saving page 6: https://www.pakwheels.com/used-cars/honda-lahore/201?page=6&_pjax=%5Bdata-pjax-container%5D
Saving page 7: https://www.pakwheels.com/used-cars/honda-lahore/201?page=7&_pjax=%5Bdata-pjax-container%5D
Saving page 8: https://www.pakwheels.com/used-cars/honda-lahore/201?page=8&_pjax=%5Bdata-pjax-container%5D
Saving page 9: https://www.pakwheels.com/used-cars/honda-lahore/201?page=9&_pjax=%5Bdata-pjax-container%5D
Saving page 10: https://www.pakwheels